# SolidWorks Float Fix Debugger
Bu notebook, aktif SolidWorks montajına bağlanır ve tüm bileşenleri 'Float' (Serbest) yapmayı dener.

In [ ]:
import win32com.client
import time

def connect_sw():
    try:
        sw_app = win32com.client.GetActiveObject("SldWorks.Application")
        print("SolidWorks'e bağlanıldı.")
        return sw_app
    except Exception as e:
        print(f"SolidWorks'e bağlanılamadı: {e}")
        print("Lütfen SolidWorks'ün açık olduğundan emin olun.")
        return None

def float_components():
    sw_app = connect_sw()
    if not sw_app: return
    
    model = sw_app.ActiveDoc
    if not model:
        print("Açık doküman yok.")
        return
        
    if model.GetType() != 2: # swDocASSEMBLY
        print("Aktif doküman bir montaj değil.")
        return
        
    print(f"Montaj: {model.GetTitle()}")
    
    # GetComponents(True) -> Top level only
    comps = model.GetComponents(True)
    if not comps:
        print("Bileşen bulunamadı.")
        return
        
    print(f"{len(comps)} bileşen taranıyor...")
    
    count = 0
    for i, comp in enumerate(comps):
        try:
            if comp.IsFixed:
                print(f"  [{i}] {comp.Name2} -> SABİT. Float yapılıyor...")
                comp.IsFixed = False
                if not comp.IsFixed:
                    print("    -> Başarılı.")
                    count += 1
                else:
                    print("    -> BAŞARISIZ.")
            else:
                # print(f"  [{i}] {comp.Name2} -> Zaten Serbest.")
                pass
        except Exception as e:
            print(f"  Hata ({comp.Name2 if comp else '?'}): {e}")
            
    if count > 0:
        print(f"{count} bileşen serbest bırakıldı. Rebuild yapılıyor...")
        model.EditRebuild3()
    else:
        print("Herhangi bir değişiklik yapılmadı.")

float_components()